In [6]:
%run packages_imported.py
%run EDA.py

In [3]:
#df_all_news_processed = pd.read_pickle(r'data/original_data/pickles/df_all_news_processed_post_dataset_distribution')

In [7]:
frame1 = df_all_news_processed.loc[df_all_news_processed['Category'] == 'entertainment'][:2000]
frame2 = df_all_news_processed.loc[df_all_news_processed['Category'] == 'financial'][:2000]
frame3 = df_all_news_processed.loc[df_all_news_processed['Category'] == 'political'][:2000]
frame4 = df_all_news_processed.loc[df_all_news_processed['Category'] == 'sport'][:2000]
frame5 = df_all_news_processed.loc[df_all_news_processed['Category'] == 'technology'][:2000]
frame6 = df_all_news_processed.loc[df_all_news_processed['Category'] == 'world'][:2000]
frame7 = df_all_news_processed.loc[df_all_news_processed['Category'] == 'travel'][:2000]

frames = [frame1, frame2, frame3, frame4, frame5, frame6, frame7]
df_processed_smaller = pd.concat(frames)
df_processed_smaller.reset_index()
df_processed_smaller.to_pickle('data/original_data/pickles/df_processed_smaller_before_tuning')

--------

In [3]:
df_all_news_processed = pd.read_pickle(r'data/original_data/pickles/df_processed_smaller_before_tuning')

In [12]:
time.time()

1606324389.4297402

In [14]:
t1 = time.time() 

#need to have a look at the functions initialisation and 'self'
EDA.process_time("", round(time.time()-t1))

Process completed.
Time taken: 0 seconds


In [ ]:
t1 = time.time() 

from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7)
X = tfidfconverter.fit_transform(df_all_news_processed['Text']).toarray()

EDA.process_time(round(time.time()-t1))

In [ ]:
t1 = time.time() 

y = df_all_news_processed['Category']

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train) 

EDA.process_time(round(time.time()-t1))